In [28]:
import numpy as np 
import pandas as pd
import gurobipy as gp 
from gurobipy import GRB

def make_problem_lp(data):
    model = gp.Model("TSP")
    n = len(data)
    
    x = model.addMVar((n,n), vtype=GRB.BINARY)
    model.addConstrs(x[i,:].sum() == 1 for i in range(n))
    model.addConstrs(x[:,j].sum() == 1 for j in range(n))
    model.addConstrs(x[i,i] == 0 for i in range(n))
    u = model.addMVar(n, vtype=GRB.INTEGER, lb = 1, ub = n-1)
    
    for i in range(1, n, 1):
        for j in range(1, n, 1):
            if i != j:
                model.addConstr(u[i] - u[j] +(n-1)*x[i,j] <= n - 2)
    
    model.setObjective(sum(x[i,j]*data[i][j] for i in range(n) for j in range(n)), GRB.MINIMIZE)
    model.setParam('TimeLimit', 500)
    model.optimize()  
    return x.X

In [29]:
import math
# На вход подаем матрицу расстояний от i-ого пункта к j-ому пункту
# locations список координат пунктов [(288, 149), (288, 129), (270, 133)]
def compute_euclidean_distance_matrix(locations):
    """Creates callback to return distance between points."""
    #print("locations: ", locations)
    distances = {}
    for from_counter, from_node in enumerate(locations):
        #print("from_counter: ", from_counter, " from_node: ", from_node)
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            if from_counter == to_counter:
                distances[from_counter][to_counter] = 0
            else:
                # Euclidean distance
                distances[from_counter][to_counter] = (int( math.hypot((from_node[0] - to_node[0]), (from_node[1] - to_node[1]))))
    return distances

In [30]:
import urllib.request

'''
c105  = https://people.idsia.ch/~luca/macs-vrptw/problems/c105.txt
c205  = https://people.idsia.ch/~luca/macs-vrptw/problems/c205.txt
r101  = https://people.idsia.ch/~luca/macs-vrptw/problems/r101.txt
r111  = https://people.idsia.ch/~luca/macs-vrptw/problems/r111.txt
r201  = https://people.idsia.ch/~luca/macs-vrptw/problems/r201.txt
rc201 = https://people.idsia.ch/~luca/macs-vrptw/problems/rc201.txt
'''
locations = []
target_url = "https://people.idsia.ch/~luca/macs-vrptw/problems/c205.txt"
line_number = 0
for line in urllib.request.urlopen(target_url):
    line_number += 1
    
    if line_number >= 9:
        CUSTOMER = [float(s) for s in line.split()]
        if len(CUSTOMER) > 0:
            locations.append((CUSTOMER[1], CUSTOMER[2]))

In [32]:
data = {}
data['distance_matrix'] = {}
data['distance_matrix'] = compute_euclidean_distance_matrix(locations)

In [33]:
x = make_problem_lp(data['distance_matrix'])

Set parameter TimeLimit to value 500
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-4460  CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 10818 rows, 10920 columns and 53254 nonzeros
Model fingerprint: 0x1918a2dd
Variable types: 0 continuous, 10920 integer (10816 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+02]
Presolve removed 104 rows and 105 columns
Presolve time: 0.10s
Presolved: 10714 rows, 10815 columns, 52942 nonzeros
Variable types: 0 continuous, 10815 integer (10712 binary)

Root relaxation: objective 4.714854e+02, 327 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

In [34]:
def print_way(x, data_points):
    num_nods = len(x)
    iteration = 1
    current_node_index = 0
    way = '0'

    while iteration != num_nods:    
        p1 = ()
        for k in range(num_nods):
            if x[current_node_index, k] == 1:
                current_node_index = k
                way += f' -> {k}'
                iteration += 1
                if len(p1) != 0:
                    p2 = (locations[k][0], locations[k][1])
                    l = (p1, p2)
                    data_points.append(l) 
                p1 = (locations[k][0], locations[k][1])
    print(way)
    
    return data_points